# Deploy to Vespa Cloud

![Vespa Cloud logo](https://cloud.vespa.ai/assets/logos/vespa-cloud-logo-full-black.png)

This notebook deploys a [Vespa.ai](https://vespa.ai/) sample application to Vespa Cloud using pyvespa.
The [Vespa CLI](https://docs.vespa.ai/en/vespa-cli.html) is used for managing security credentials.

See pyvespa, and jupyter notebook [requirements](https://pyvespa.readthedocs.io/en/latest/index.html#requirements)
to run this notebook.

Refer to the [sample apps](https://github.com/vespa-engine/sample-apps) site for more applications.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vespa-engine/pyvespa/blob/master/docs/sphinx/source/deploy-vespa-cloud.ipynb)

## Install pyvespa and the Vespa CLI

In [ ]:
!pip install pyvespa

If homebrew is installed, install the Vespa CLI:

In [ ]:
!brew install vespa-cli

Alternatively, when running in Colab, install the Vespa CLI:

In [ ]:
import os
import requests
res = requests.get(url="https://api.github.com/repos/vespa-engine/vespa/releases/latest").json()
os.environ["VERSION"] = res["tag_name"].replace("v", "")
!curl -fsSL https://github.com/vespa-engine/vespa/releases/download/v${VERSION}/vespa-cli_${VERSION}_linux_amd64.tar.gz | tar -zxf -
!ln -sf /content/vespa-cli_${VERSION}_linux_amd64/bin/vespa /usr/local/bin/vespa

## Create a Vespa Cloud tenant

Sign up and create a tenant: Run steps 1 and 2 in [getting started](https://cloud.vespa.ai/en/getting-started), then replace the tenant name below:

In [ ]:
import os
os.environ["TENANT_NAME"] = "mytenant"

## Set up security credentials
Configure the tenant.application.instance:

In [ ]:
!vespa config set target cloud
!vespa config set application ${TENANT_NAME}.myapp.default

If the tenant.application.instance is already created with security credentials, skip next two steps, and go directly to setting `api_key_path`. If not, create data-plane credentials:

In [ ]:
!vespa auth cert -N

`vespa auth cert -N` stores the credentials in ~/.vespa, like:

      /Users/me/.vespa/mytenant.myapp.default/data-plane-public-cert.pem
      /Users/me/.vespa/mytenant.myapp.default/data-plane-private-key.pem

Create an api-key, used to deploy the application:

In [ ]:
!vespa auth api-key

`vespa auth api-key` stores the api key in ~/.vespa, like:

      /Users/me/.vespa/mytenant.api-key.pem

Add it in the console per instructions from the command. Enter full path to the api key:

In [ ]:
api_key_path = "/Users/me/.vespa/mytenant.api-key.pem"

## Create an application package

An application package is the application's configuration, including schema.
Here, using an empty appliction to ilustrate deployment only -
see the other examples in pyvespa for real application packages.

In [ ]:
from vespa.package import ApplicationPackage

app = ApplicationPackage(name="sampleapp")

## Create a VespaCloud instance

Refer to the [VespaCloud reference](reference-api.rst#vespacloud).

In [ ]:
from vespa.deployment import VespaCloud

vespa_cloud = VespaCloud(
    tenant=os.getenv("TENANT_NAME"),
    application="myapp",
    key_location=api_key_path,
    application_package=app)

## Deploy to Vespa Cloud

Refer to [tenants, applications and instances](https://cloud.vespa.ai/en/tenant-apps-instances) for details - here creating an instance named `default`:

In [ ]:
app = vespa_cloud.deploy(instance="default")

That is it, you can now interact with your deployed application through the `app` instance.